In [205]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import numpy as np
from PIL import Image
import pandas as pd
import os 
import matplotlib.pyplot as plt
import math
import random
import sklearn

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
words=open('names.txt').read().splitlines()

In [3]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
for w in words[:1]:
    chs=['<S>']+list(w)+['<E>']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        print(ch1,ch2,ch3)
        

<S> e m
e m m
m m a
m a <E>


In [6]:
b={}
for w in words:
    chs=['<S>']+list(w)+['<E>']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        # print(chs,chs[1:],chs[2:])
        bigram=(ch1,ch2,ch3)
        b[bigram]=b.get(bigram,0)+1

In [7]:
chars=sorted(list(set(''.join(words))))

In [8]:
s_to_i={s:i+1 for i,s in enumerate(chars)}
s_to_i['.']=0

s_to_i

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [169]:
x=[]
y=[]

for w in words:
    chs=['.']+list(w)+['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1=s_to_i[ch1]
        ix2=s_to_i[ch2]
        ix3=s_to_i[ch3]
        x.append([ix1,ix2])
        y.append(ix3)
 
x=torch.tensor(x,dtype=torch.int64).to(device)
y=torch.tensor(y,dtype=torch.int64).to(device)

In [170]:
import torch.nn.functional as F
xenc=F.one_hot(x,num_classes=27).float().to(device)

In [171]:
g=torch.Generator().manual_seed(2147483647)

In [172]:
w=torch.randn((27*2,27),requires_grad=True,device=device)
xenc=xenc.reshape(-1,2*27).to(device)
xenc.shape

torch.Size([196113, 54])

In [177]:
logits = xenc @ w
print(logits.shape)
counts = torch.exp(logits)
probs = counts / counts.sum(dim = 1, keepdim = True)
loss = - probs[torch.arange(len(x)), y].log().mean()
loss

torch.Size([196113, 27])


tensor(3.0766, device='cuda:0', grad_fn=<NegBackward0>)

In [176]:
w.grad=None #zero_grad
loss.backward()

w.data-=50*w.grad

In [179]:
#pipeline 
x=[]
y=[]

for w in words:
    chs=['.']+list(w)+['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1=s_to_i[ch1]
        ix2=s_to_i[ch2]
        ix3=s_to_i[ch3]
        x.append([ix1,ix2])
        y.append(ix3)
 
x=torch.tensor(x).to(device)
y=torch.tensor(y).to(device)
w=torch.randn((27*2,27),requires_grad=True,device=device)

In [202]:
for i in range(300):
    xenc=F.one_hot(x,num_classes=27).float().to(device)
    xenc=xenc.reshape(-1,2*27).to(device)
    logits= xenc @ w
    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)
    loss = - probs[torch.arange(len(x)), y].log().mean()
    #  # add regularization
    # loss += 0.2 * W.pow(2).mean()
    if i%10==0:
        print(loss.item())
    w.grad=None
    loss.backward()
    w.data+=-50*w.grad
    
    

2.237687110900879
2.237675905227661
2.2376649379730225
2.237654447555542
2.2376437187194824
2.237633228302002
2.2376229763031006
2.237612724304199
2.237602949142456
2.237593173980713
2.2375833988189697
2.2375738620758057
2.2375645637512207
2.2375552654266357
2.23754620552063
2.237537145614624
2.2375285625457764
2.2375199794769287
2.237511396408081
2.2375028133392334
2.237494468688965
2.2374861240386963
2.237478256225586
2.2374701499938965
2.237462282180786
2.237454414367676
2.2374467849731445
2.2374396324157715
2.2374320030212402
2.237424612045288


In [203]:
names = []
i_to_s={i:s for s,i in s_to_i.items()}
for i in range(10):
    out = []
    idx1, idx2 = 0, 0
    while True:

        # now we use the softmax of the logits
        xenc = F.one_hot(torch.tensor([idx1, idx2]).to(device), num_classes = 27).float().to(device)
        xenc = xenc.view(-1, 27*2)
        # print(xenc.shape)
        logits = xenc @ w
        counts = torch.exp(logits)
        p = counts / counts.sum(dim = 1, keepdim = True)

        idx1 = idx2
        idx2 = torch.multinomial(p.to(device), num_samples = 1 , replacement = True).item()
        out.append(i_to_s[idx2])
        if idx2 == 0:
            break

    names.append("".join(out))
    
for name in names:
    print(name)

lanton.
elyn.
bran.
jlana.
ynyes.
sam.
mana.
nabera.
lalin.
zersisliy.
